# 1 - PROBLEM

## Requisitos

[OK] 1. **Leitura de Dados:** Desenvolva um componente capaz de ler uma lista de objetos contidos em um arquivo JSON, representando informações sobre tema escolhidoes.

[Falta redução] 2. **Mapeamento, Filtro e Redução:** Implemente funcionalidades para realizar mapeamento, filtragem e redução dos dados, proporcionando uma análise mais refinada das informações contidas no conjunto de dados.

[OK] 3. **Manipulação de Dados Individuais:** Permita a leitura individual, atualização e exclusão de do arquivos, mantendo o arquivo JSON sempre atualizado.

[OK, dentro do possível] 4. **Validações de Operações:** Integre validações utilizando blocos try-except e raise para garantir a robustez das operações, prevenindo erros e assegurando a consistência dos dados.

[OK] 5. **Obtenção de Estatísticas Simples:** Desenvolva uma função para extrair dados estatísticos simples, como média, máximo e mínimo, por exemplo, sum item X nos dados de exatas.

[Falta] 6. **Identificação de Máximos/Mínimos com Detalhes:** Crie uma função que retorne uma lista de tuplas, contendXdo professor e o valor máximo (ou mínimo) de algum atributo numérico. Esta função deve ser configurável para fornecer estatísticas de máximo ou mínimo.

[OK] 7. **Exportação de Dados Estatísticos para CSV:** Implemente a capacidade de salvar os dados estatísticos obtidos em um arquivo CSV, permitindo uma análise posterior ou compartilhamento fácil dos resultados.

# 2 - IMPORTS

## 2.1 - Bibliotecas

In [1]:
import json
from funcoes import obter_opcoes
import csv
from IPython.display import clear_output

## 2.2 - Funções

### 2.2.1 - Leitura do Arquivo

In [2]:
def carrega_dados(path:str = 'data/receitas.json') -> list[dict]:
    try:
        with open(path, 'r', encoding='utf-8') as arquivo:
            dados = arquivo.read()
            return json.loads(dados)
    except FileNotFoundError:
        return []

### 2.2.2 - Salvamento do Arquivo

In [3]:
def salvar_dados(dados:list[dict], path:str = 'data/receitas.json') -> bool:
    try:
        with open(path, 'w', encoding='utf-8') as arquivo:
            arquivo.write(json.dumps(dados))
            return True
    except Exception:
        return False

### 2.2.3 - Visualização do Arquivo

In [4]:
def formata_dicionario(dados: dict) -> str:
    return ' | '.join([f'{k}: {v}' for k, v in dados.items()])

In [5]:
def formata_ingredientes(ingredientes: list) -> str:
    return '\n  '.join(map(formata_dicionario, ingredientes))

In [6]:
def formata_receita(receita):
    nome = receita['nome']
    ingredientes = formata_ingredientes(receita['ingredientes'])
    instrucoes = receita['instrucoes']
    return f'{nome}:\n Ingredientes:\n  {ingredientes}\n Instruções: {instrucoes}\n'

In [7]:
def formata_lista_receitas(lista_receitas):
    return '\n'.join(map(formata_receita, lista_receitas))

### 2.2.4 - Adiciona Elemento

In [8]:
def obter_nome():
    while True:
        nome = input("Digite o nome da receita: ").strip().title()
        if nome:
            return nome
        else:
            print("Nome inválido. Por favor, digite novamente.")

In [9]:
def obter_ingrediente():
    while True:
        ingrediente = input("Digite o nome do ingrediente (ou 'fim' para encerrar): ").strip().title()
        if ingrediente.lower() == 'fim' or ingrediente:
            return ingrediente
        else:
            print("Ingrediente inválido. Por favor, digite novamente.")

In [10]:
def obter_quantidade():
    while True:
        quantidade = input("Digite a quantidade do ingrediente e sua unidade: ")
        return quantidade

In [11]:
def obter_custo():
    while True:
        custo = input("Digite o valor do ingrediente: ")
        if custo and validar_numero(custo):
            return float(custo)
        else:
            print("Custo inválido. Por favor, digite novamente.")

In [12]:
def validar_numero(valor):
    try:
        return float(valor) >= 0
    except ValueError:
        return False

In [13]:
def obter_instrucao():
    return input("Digite a instrução de preparo da receita: ").capitalize()

In [14]:
def criar_nova_receita(dados: list[dict]) -> bool:
    nova_receita = {'nome': obter_nome(), 'ingredientes': []}

    while True:
        ingrediente = obter_ingrediente()
        if ingrediente.lower() == 'fim':
            break

        quantidade = obter_quantidade()
        custo = obter_custo()

        nova_receita['ingredientes'].append({
            'nome': ingrediente,
            'quantidade': quantidade,
            'custo': custo
        })

    nova_receita['instrucoes'] = obter_instrucao()

    dados.append(nova_receita)
    return True

### 2.2.5 - Buscar Elementos


In [15]:
def busca(data: list[dict], input_name: str, field='nome') -> list[dict]:    
    return list(filter(lambda item: item[field].strip().title() == input_name.strip().title(), data))

In [16]:
def verifica_busca(data: list[dict]):    
    if not data:
        print(f'\nNão encontrado.\n')
    return data

### 2.2.6 - Alterar Elementos

In [17]:
def executa_alteracao(opc, opc_funcao, dado: dict) -> None:
    while True:
        opcao = obter_opcoes(opc, 'Escolha o campo a ser alterado')

        if opcao == 'F':
            break
        opc_funcao[opcao][1](dado)

In [18]:
def atualizar_nome_ingrediente(ingredientes):
    ingredientes['ingrediente'] = obter_ingrediente()

def atualizar_quantidade(ingredientes):
    ingredientes['quantidade'] = obter_quantidade()

def atualizar_custo(ingredientes):
    ingredientes['custo'] = obter_custo()

In [19]:
def alterar_receita(receitas: list[dict]) -> bool:
    alterado = verifica_busca(busca(receitas, obter_nome(), 'nome'))

    if not alterado:        
        return False

    alterado = alterado[0]

    opc = {
        'N': 'Nome',
        'I': 'Ingredientes',
        'P': 'Instruções de Preparo',
        'F': 'Finalizar'
    }

    opc_funcao = {
        'N' : ('nome', atualizar_nome),
        'I': ('ingredientes', atualizar_ingrediente),
        'P': ('instrucoes', atualizar_instrucao),
    }

    msg = f'Tem certeza que deseja alterar [{formata_receita(alterado)}]'

    if obter_opcoes({'S': 'Sim', 'N': 'Não'}, msg) == 'S':
        executa_alteracao(opc, opc_funcao, alterado)
        return True
    else:
        return False

In [20]:
def atualizar_nome(receita: dict):
    receita['nome'] = obter_nome()

def atualizar_instrucao(receita: dict):
    receita['instrucoes'] = obter_instrucao()

def atualizar_ingrediente(receita: dict) -> bool:
    opc = {
        'I': 'Ingrediente',
        'Q': 'Quantidade',
        'C': 'Custo',
        'F': 'Finalizar'
    }

    opc_funcao = {
        'I' : ('ingrediente', atualizar_nome_ingrediente),
        'Q': ('quantidade', atualizar_quantidade),
        'C': ('custo', atualizar_custo)
    }
    ingredientes = receita['ingredientes']
    alterado = verifica_busca(busca(ingredientes, obter_ingrediente(), 'ingrediente'))

    if not alterado:
        return False

    msg = f'Tem certeza que deseja alterar [{formata_ingredientes(alterado)}]'

    alterado = alterado[0]

    if obter_opcoes({'S': 'Sim', 'N': 'Não'}, msg) == 'S':
        executa_alteracao(opc, opc_funcao, alterado)
        return True
    else:
        return False

### 2.2.7 - Deletar

In [21]:
def deletar_ingrediente(receitas: list[dict]) -> bool:
    receita = verifica_busca(busca(receitas, obter_nome(), 'nome'))
    if not receita:
        return False
        
    ingredientes = receita[0]['ingredientes']
    apagado = verifica_busca(busca(ingredientes, obter_ingrediente(), 'ingrediente'))
    
    if not apagado:        
        return False
    
    msg = f'Tem certeza que deseja excluir [{formata_ingredientes(apagado)}]'
    
    apagado = apagado[0]

    if obter_opcoes({'S': 'Sim', 'N': 'Não'}, msg) == 'S':
        ingredientes.remove(apagado)
        return True
    else:
        return False

In [22]:
def deletar_receita(receitas: list[dict]) -> bool:
    apagado = verifica_busca(busca(receitas, obter_nome(), 'nome'))
    
    if not apagado:        
        return False
    
    apagado = apagado[0]
    
    msg = f'Tem certeza que deseja excluir [{formata_receita(apagado)}]'
    
    if obter_opcoes({'S': 'Sim', 'N': 'Não'}, msg) == 'S':
        receitas.remove(apagado)
        return True
    else:
        return False

### 2.2.8 - Calculando Custos

In [23]:
def calcula_custo(receita):
    custos_por_receita = []
    for item in receita:
        custo = round(sum(ingrediente["custo"] for ingrediente in item["ingredientes"]), 2)
        nome = item["nome"]
        custos_por_receita.append((nome, custo))
    return custos_por_receita

In [24]:
def receita_mais_cara(data):
    maior_preco = max(data, key=lambda x: x[1])
    return [item for item in data if item[1] >= maior_preco[1]]

In [25]:
def receita_mais_barata(data):
    menor_preco = min(data, key=lambda x: x[1])
    return [item for item in data if item[1] <= menor_preco[1]]

### 2.2.9 - Salvar Dados Estatístico em um CSV

In [26]:
def salvar_dados_estatistico(dados) -> bool:
    custo = calcula_custo(dados)    
    mais_cara = receita_mais_cara(custo)
    mais_barata = receita_mais_barata(custo)

    tabela = [custo, mais_cara, mais_barata]
    
    # cria o arquivo CSV
    arquivo = open('Dados Estatisticos.csv', 'w', encoding='utf-8-sig')

    # definindo as regras do nosso CSV:
    # ele será escrito no arquivo apontado pela variável 'arquivo'
    # seus elementos serão delimitados (delimiter) pelo símbolo ';'
    # suas linhas serão encerradas (lineterminator) por uma quebra de linha
    escritor = csv.writer(arquivo, delimiter=';', lineterminator='\n')

    # escreve uma lista de listas em formato CSV:
    escritor.writerows(tabela)

    # fecha e salva o arquivo
    arquivo.close()

    return True

### 2.2.10 - Visualizar

In [33]:
def exibir_receita(dados):
    receita = formata_lista_receitas(verifica_busca(busca(dados, obter_nome())))
    print(f'\n{receita}')

In [28]:
def exibir_todas_receitas(dados):
    print(formata_lista_receitas(dados))

# 3 - Menu Principal

In [29]:
def sair(dados):
    pass

In [30]:
opc = {
    '1': 'Criar nova receita\n', 
    '2': 'Alterar receita\n', 
    '3': 'Deletar receita\n', 
    '4': 'Deletar ingrediente\n', 
    '5': 'Buscar\n', 
    '6': 'Exibir Todos\n',
    '7': 'Obter Dados Estatísticos .csv\n',
    '8': 'Sair\n'
}

opc_func = {
    '1': criar_nova_receita,
    '2': alterar_receita,
    '3': deletar_receita,
    '4': deletar_ingrediente,
    '5': exibir_receita,
    '6': exibir_todas_receitas,
    '7': salvar_dados_estatistico,
    '8': sair
}

In [32]:
saida = False
while not saida:
    dados = carrega_dados()
    opcao = obter_opcoes(opc, '\n==================== MENU ====================')
    opc_func[opcao](dados)
    
    salvar_dados(dados)

    if opcao == '8':
        saida = True
else:
    print('\nObrigado por utilizar o software')


==================== MENU ==================== 
 1 - Criar nova receita
 2 - Alterar receita
 3 - Deletar receita
 4 - Deletar ingrediente
 5 - Buscar
 6 - Exibir Todos
 7 - Obter Dados Estatísticos .csv
 8 - Sair
 Opções: 8



Obrigado por utilizar o software
